In [59]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [42]:
eda_gas_df = pd.read_csv("data/PROCESSED_heating-gas-consumption-and-cost.csv")
eda_gas_df

,Development Name,Borough,Location,Funding Source,Vendor Name,Revenue Month,Service Start Date,Service End Date,# days,Current Charges,Consumption (Therms),ES Commodity
0,ADAMS,BRONX,BLD 04,FEDERAL,CONSOLIDATED EDISON COMPANY OF NY,2010-01-01,2009-12-24,2010-01-26,33,78292.97,136632.00,UTILITY GAS
1,ADAMS,BRONX,BLD 04,FEDERAL,HESS CORPORATION (905243) - 2,2010-01-01,2009-12-25,2010-01-26,32,133779.54,136632.00,BROKERED GAS
2,ALBANY/ALBANY II,BROOKLYN,ALBANY BLD 04,FEDERAL,National Grid NYC,2010-01-01,2009-12-29,2010-01-28,30,44335.21,153899.18,UTILITY GAS
3,ALBANY/ALBANY II,BROOKLYN,ALBANY BLD 04,FEDERAL,National Grid NYC,2010-01-01,2009-12-29,2010-01-28,30,22034.86,76488.82,UTILITY GAS
4,AMSTERDAM,MANHATTAN,BLD 02,FEDERAL,CONSOLIDATED EDISON COMPANY OF NY,2010-01-01,2009-12-24,2010-01-26,33,95456.57,169137.00,UTILITY GAS
...,...,...,...,...,...,...,...,...,...,...,...,...
179590,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,DIRECT ENERGY BUSINESS (32179),2021-04-01,2021-03-27,2021-04-27,31,23805.14,56991.00,BROKERED GAS
179591,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,DIRECT ENERGY BUSINESS (32179),2021-05-01,2021-04-28,2021-05-25,27,20283.93,48561.00,BROKERED GAS
179592,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,DIRECT ENERGY BUSINESS (32179),2021-06-01,2021-05-26,2021-06-25,30,1509.99,3615.00,BROKERED GAS
179593,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,DIRECT ENERGY BUSINESS (32179),2021-12-01,2021-07-28,2022-01-06,162,184610.42,435135.10,BROKERED GAS


In [43]:
eda_gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179595 entries, 0 to 179594
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Development Name      179595 non-null  object 
 1   Borough               179595 non-null  object 
 2   Location              179595 non-null  object 
 3   Funding Source        179595 non-null  object 
 4   Vendor Name           179595 non-null  object 
 5   Revenue Month         179595 non-null  object 
 6   Service Start Date    179595 non-null  object 
 7   Service End Date      179595 non-null  object 
 8   # days                179595 non-null  int64  
 9   Current Charges       179595 non-null  float64
 10  Consumption (Therms)  179595 non-null  float64
 11  ES Commodity          179595 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 16.4+ MB


In [44]:
eda_gas_df['Revenue Month'] = pd.to_datetime(eda_gas_df['Revenue Month'], format="%Y-%m")
eda_gas_df['Service Start Date'] = eda_gas_df['Service Start Date'].astype("datetime64")
eda_gas_df['Service End Date'] = eda_gas_df['Service End Date'].astype("datetime64")

In [45]:
eda_gas_df["Year"] = eda_gas_df["Revenue Month"].dt.year

In [46]:
category_df = eda_gas_df.loc[:, ['Development Name', 'Borough', 'Location', 'Vendor Name']]
one_hot_df = pd.get_dummies(category_df)
one_hot_df

,Development Name_1010 EAST 178TH STREET,Development Name_104-14 TAPSCOTT STREET,Development Name_1162-1176 WASHINGTON AVENUE,Development Name_131 SAINT NICHOLAS AVENUE,Development Name_1471 WATSON AVENUE,Development Name_154 WEST 84TH STREET,Development Name_303 VERNON AVENUE,Development Name_335 EAST 111TH STREET,Development Name_45 ALLEN STREET,Development Name_572 WARREN STREET,...,Vendor Name_CONSOLIDATED EDISON COMPANY OF NY,Vendor Name_DIRECT ENERGY BUSINESS (32179),Vendor Name_DIRECT ENERGY BUSINESS (905243),Vendor Name_EAST COAST POWER & GAS,Vendor Name_EAST COAST POWER AND GAS (MRI),Vendor Name_HESS CORPORATION (905243) - 2,Vendor Name_Hudson Energy Services LI,Vendor Name_Hudson Energy ServicesB,Vendor Name_National Grid LI,Vendor Name_National Grid NYC
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179590,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179591,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179592,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179593,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [47]:
dummy_df = pd.get_dummies(category_df, drop_first=True)
dummy_df

,Development Name_104-14 TAPSCOTT STREET,Development Name_1162-1176 WASHINGTON AVENUE,Development Name_131 SAINT NICHOLAS AVENUE,Development Name_1471 WATSON AVENUE,Development Name_154 WEST 84TH STREET,Development Name_303 VERNON AVENUE,Development Name_335 EAST 111TH STREET,Development Name_45 ALLEN STREET,Development Name_572 WARREN STREET,Development Name_830 AMSTERDAM AVENUE,...,Location_WEST BRIGHTON I BLD 06,Vendor Name_DIRECT ENERGY BUSINESS (32179),Vendor Name_DIRECT ENERGY BUSINESS (905243),Vendor Name_EAST COAST POWER & GAS,Vendor Name_EAST COAST POWER AND GAS (MRI),Vendor Name_HESS CORPORATION (905243) - 2,Vendor Name_Hudson Energy Services LI,Vendor Name_Hudson Energy ServicesB,Vendor Name_National Grid LI,Vendor Name_National Grid NYC
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179590,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179591,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179592,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
179593,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [48]:
categorical_cols = ['Development Name', 'Borough', 'Location', 'Vendor Name']
label_category_df = pd.DataFrame()
for col in categorical_cols:
    label_category_df[col] = category_df[col]
    label_category_df[col + '_Label_Encoded'], _ = pd.factorize(category_df[col]) 

label_category_df

,Development Name,Development Name_Label_Encoded,Borough,Borough_Label_Encoded,Location,Location_Label_Encoded,Vendor Name,Vendor Name_Label_Encoded
0,ADAMS,0,BRONX,0,BLD 04,0,CONSOLIDATED EDISON COMPANY OF NY,0
1,ADAMS,0,BRONX,0,BLD 04,0,HESS CORPORATION (905243) - 2,1
2,ALBANY/ALBANY II,1,BROOKLYN,1,ALBANY BLD 04,1,National Grid NYC,2
3,ALBANY/ALBANY II,1,BROOKLYN,1,ALBANY BLD 04,1,National Grid NYC,2
4,AMSTERDAM,2,MANHATTAN,2,BLD 02,2,CONSOLIDATED EDISON COMPANY OF NY,0
...,...,...,...,...,...,...,...,...
179590,WYCKOFF GARDENS,297,BROOKLYN,1,BLD 03,12,DIRECT ENERGY BUSINESS (32179),7
179591,WYCKOFF GARDENS,297,BROOKLYN,1,BLD 03,12,DIRECT ENERGY BUSINESS (32179),7
179592,WYCKOFF GARDENS,297,BROOKLYN,1,BLD 03,12,DIRECT ENERGY BUSINESS (32179),7
179593,WYCKOFF GARDENS,297,BROOKLYN,1,BLD 03,12,DIRECT ENERGY BUSINESS (32179),7


In [49]:
count_encode_df = pd.DataFrame()
for col in categorical_cols:
    counts = category_df[col].value_counts().sort_values(ascending=False)
    count_encode_df[col] = category_df[col]
    count_encode_df[col + '_Count_Encoded'] = count_encode_df[col].map(counts)

count_encode_df

,Development Name,Development Name_Count_Encoded,Borough,Borough_Count_Encoded,Location,Location_Count_Encoded,Vendor Name,Vendor Name_Count_Encoded
0,ADAMS,260,BRONX,45148,BLD 04,15838,CONSOLIDATED EDISON COMPANY OF NY,48299
1,ADAMS,260,BRONX,45148,BLD 04,15838,HESS CORPORATION (905243) - 2,20685
2,ALBANY/ALBANY II,156,BROOKLYN,72708,ALBANY BLD 04,472,National Grid NYC,42998
3,ALBANY/ALBANY II,156,BROOKLYN,72708,ALBANY BLD 04,472,National Grid NYC,42998
4,AMSTERDAM,281,MANHATTAN,46202,BLD 02,19568,CONSOLIDATED EDISON COMPANY OF NY,48299
...,...,...,...,...,...,...,...,...
179590,WYCKOFF GARDENS,209,BROOKLYN,72708,BLD 03,15327,DIRECT ENERGY BUSINESS (32179),39881
179591,WYCKOFF GARDENS,209,BROOKLYN,72708,BLD 03,15327,DIRECT ENERGY BUSINESS (32179),39881
179592,WYCKOFF GARDENS,209,BROOKLYN,72708,BLD 03,15327,DIRECT ENERGY BUSINESS (32179),39881
179593,WYCKOFF GARDENS,209,BROOKLYN,72708,BLD 03,15327,DIRECT ENERGY BUSINESS (32179),39881


In [50]:
numeric_df = eda_gas_df[eda_gas_df['ES Commodity'] == 'UTILITY GAS'].loc[:, ['Current Charges', 'Consumption (Therms)']]
numeric_df = numeric_df.reset_index(drop=True)
numeric_df

,Current Charges,Consumption (Therms)
0,78292.97,136632.00
1,44335.21,153899.18
2,22034.86,76488.82
3,95456.57,169137.00
4,14607.73,26301.00
...,...,...
92194,1419.26,3615.00
92195,1036.26,1619.00
92196,2505.80,6768.00
92197,20992.58,50105.00


In [51]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)
standard_scale_df = pd.DataFrame(scaled_data, columns=['Current Charges_Standard_Scale', 'Consumption (Therms)_Standard_Scale'])
standard_scale_df

,Current Charges_Standard_Scale,Consumption (Therms)_Standard_Scale
0,2.668961,2.346788
1,1.341611,2.696895
2,0.469929,1.127332
3,3.339856,3.005855
4,0.179615,0.109730
...,...,...
92194,-0.335900,-0.350248
92195,-0.350870,-0.390719
92196,-0.293429,-0.286318
92197,0.429188,0.592377


In [52]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_df)
minmax_scale_df = pd.DataFrame(scaled_data, columns=[ 'Current Charges_MinMax_Scale', 'Consumption (Therms)_MinMax_Scale'])
minmax_scale_df

,Current Charges_MinMax_Scale,Consumption (Therms)_MinMax_Scale
0,0.084319,0.058100
1,0.047747,0.065443
2,0.023731,0.032525
3,0.102804,0.071922
4,0.015732,0.011184
...,...,...
92194,0.001528,0.001537
92195,0.001116,0.000688
92196,0.002698,0.002878
92197,0.022608,0.021306


In [53]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(numeric_df)
robust_scale_df = pd.DataFrame(scaled_data, columns=['Current Charges_Robust_Scale', 'Consumption (Therms)_Robust_Scale'])
robust_scale_df

,Current Charges_Robust_Scale,Consumption (Therms)_Robust_Scale
0,13.037723,9.836272
1,7.284306,11.101626
2,3.505988,5.428927
3,15.945729,12.218267
4,2.247620,1.751120
...,...,...
92194,0.013115,0.088670
92195,-0.051776,-0.057599
92196,0.197206,0.319724
92197,3.329396,3.495498


In [58]:
pca = PCA(n_components=None, svd_solver="auto")
numeric_df_pca = pca.fit_transform(numeric_df)
numeric_df_pca = pd.DataFrame(numeric_df_pca, columns=['Current Charges_PCA', 'Consumption (Therms)_PCA'])
# In kết quả
numeric_df_pca

,Current Charges_PCA,Consumption (Therms)_PCA
0,134034.314963,9665.460170
1,134393.191175,-28428.583685
2,55143.611816,-13965.747970
3,170781.176027,10577.815342
4,6891.016688,1708.007157
...,...,...
92194,-19293.552690,-12.069421
92195,-21251.563885,532.813281
92196,-15986.357883,-441.534996
92197,31050.353385,-3162.637143


In [72]:
kmeans = KMeans(n_clusters=3, n_init=1)
df_clustered = kmeans.fit_predict(numeric_df)
kmeans_df = numeric_df.copy()
kmeans_df['Cluster'] = df_clustered
kmeans_df

,Current Charges,Consumption (Therms),Cluster
0,78292.97,136632.00,0
1,44335.21,153899.18,0
2,22034.86,76488.82,0
3,95456.57,169137.00,1
4,14607.73,26301.00,2
...,...,...,...
92194,1419.26,3615.00,2
92195,1036.26,1619.00,2
92196,2505.80,6768.00,2
92197,20992.58,50105.00,0


In [78]:
discretized_df = pd.DataFrame()
for col in numeric_df.columns:
    discretized_df[col] = numeric_df[col]
    discretized_df[col + "_Discretized"] = pd.qcut(numeric_df[col], 3)
discretized_df

,Current Charges,Current Charges_Discretized,Consumption (Therms),Consumption (Therms)_Discretized
0,78292.97,"(3352.53, 928530.45]",136632.00,"(6590.333, 2351667.0]"
1,44335.21,"(3352.53, 928530.45]",153899.18,"(6590.333, 2351667.0]"
2,22034.86,"(3352.53, 928530.45]",76488.82,"(6590.333, 2351667.0]"
3,95456.57,"(3352.53, 928530.45]",169137.00,"(6590.333, 2351667.0]"
4,14607.73,"(3352.53, 928530.45]",26301.00,"(6590.333, 2351667.0]"
...,...,...,...,...
92194,1419.26,"(620.92, 3352.53]",3615.00,"(1218.0, 6590.333]"
92195,1036.26,"(620.92, 3352.53]",1619.00,"(1218.0, 6590.333]"
92196,2505.80,"(620.92, 3352.53]",6768.00,"(6590.333, 2351667.0]"
92197,20992.58,"(3352.53, 928530.45]",50105.00,"(6590.333, 2351667.0]"


In [93]:
binning_df = pd.DataFrame()
for col in numeric_df.columns:
    binning_df[col] = numeric_df[col]
    binning_df[col + "_Binning"] = pd.cut(numeric_df[col], [0,
                                                            np.quantile(numeric_df[col], 0.25),
                                                            np.quantile(numeric_df[col], 0.5),
                                                            np.quantile(numeric_df[col], 0.75),
                                                            np.inf])
binning_df

,Current Charges,Current Charges_Binning,Consumption (Therms),Consumption (Therms)_Binning
0,78292.97,"(6341.27, inf]",136632.00,"(14441.125, inf]"
1,44335.21,"(6341.27, inf]",153899.18,"(14441.125, inf]"
2,22034.86,"(6341.27, inf]",76488.82,"(14441.125, inf]"
3,95456.57,"(6341.27, inf]",169137.00,"(14441.125, inf]"
4,14607.73,"(6341.27, inf]",26301.00,"(14441.125, inf]"
...,...,...,...,...
92194,1419.26,"(1341.85, 6341.27]",3615.00,"(2405.0, 14441.125]"
92195,1036.26,"(439.08, 1341.85]",1619.00,"(795.0, 2405.0]"
92196,2505.80,"(1341.85, 6341.27]",6768.00,"(2405.0, 14441.125]"
92197,20992.58,"(6341.27, inf]",50105.00,"(14441.125, inf]"
